In [2]:
import kaggle 
!kaggle datasets download -d taeefnajib/used-car-price-prediction-dataset

Dataset URL: https://www.kaggle.com/datasets/taeefnajib/used-car-price-prediction-dataset
License(s): Attribution 4.0 International (CC BY 4.0)




  0%|          | 0.00/109k [00:00<?, ?B/s]
100%|██████████| 109k/109k [00:00<00:00, 160kB/s]
100%|██████████| 109k/109k [00:00<00:00, 160kB/s]


In [3]:
import zipfile
zip = zipfile.ZipFile('used-car-price-prediction-dataset.zip')
zip.extractall()
zip.close()

# Import Lib

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [3]:
df = pd.read_csv('used_cars.csv')

**Brand & Model:** Identify the brand or company name along with the specific model of each vehicle.  

**Model Year:** Discover the manufacturing year of the vehicles, crucial for assessing depreciation and technology advancements.  

**Mileage:** Obtain the mileage of each vehicle, a key indicator of wear and tear and potential maintenance requirements.  

**Fuel Type:** Learn about the type of fuel the vehicles run on, whether it's gasoline, diesel, electric, or hybrid.  

**Engine Type:** Understand the engine specifications, shedding light on performance and efficiency.  

**Transmission:** Determine the transmission type, whether automatic, manual, or another variant.  

**Exterior & Interior Colors:** Explore the aesthetic aspects of the vehicles, including exterior and interior color options.  

**Accident History:** Discover whether a vehicle has a prior history of accidents or damage, crucial for informed decision-making.  

**Clean Title:** Evaluate the availability of a clean title, which can impact the vehicle's resale value and legal status.  

**Price:** Access the listed prices for each vehicle, aiding in price comparison and budgeting.  

In [49]:
df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


# Data Cleaning

In [5]:
df['clean_title'].unique()

array(['Yes', nan], dtype=object)

In [41]:
df['price'] = df['price'].str.extract('(\d+,\d+)', expand=True)
df['price'] = df['price'].str.replace(',', '').astype(int)

In [43]:
df['milage'] =df['milage'].str.replace(' mi.', '')
df['milage'] = df['milage'].str.replace(',', '').astype(int)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [8]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
916,Porsche,911 Carrera 4S,2018,"31,678 mi.",Gasoline,3.0 Liter,–,Silver,–,None reported,NaN,"$106,798"
26,Nissan,Sentra SR,2016,"96,541 mi.",Gasoline,1.8 Liter,Automatic,Blue,Black,None reported,NaN,"$13,998"
3656,Dodge,Durango SLT,2004,"143,000 mi.",Gasoline,335.0HP 5.7L 8 Cylinder Engine Gasoline Fuel,A/T,Beige,Beige,None reported,Yes,"$3,200"
3415,Audi,A6 2.0T Premium Plus,2016,"96,000 mi.",Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,8-Speed A/T,Black,Black,None reported,Yes,"$21,990"
3654,BMW,M235 i,2015,"68,750 mi.",Gasoline,320.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Red,Black,None reported,Yes,"$25,999"


## Mising Values

In [11]:
df.isnull().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

In [7]:
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', nan, 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [10]:
df.groupby('fuel_type')['fuel_type'].count()

fuel_type
Diesel             116
E85 Flex Fuel      139
Gasoline          3309
Hybrid             194
Plug-In Hybrid      34
not supported        2
–                   45
Name: fuel_type, dtype: int64

In [14]:
df['fuel_type'] = df['fuel_type'].fillna(df['fuel_type'].mode()[0])

In [9]:
df['brand'].nunique()

57

In [10]:
df.groupby('brand')['price'].mean().sort_values(ascending=False).head(10)

brand
Rolls-Royce    370992.727273
Lamborghini    291233.846154
Ferrari        243790.666667
McLaren        213457.500000
Bentley        137553.545455
Aston          115199.555556
Lucid          101966.333333
Rivian          93138.176471
Porsche         80804.029851
Maybach         64250.000000
Name: price, dtype: float64

In [17]:
df['accident'].unique()

array(['At least 1 accident or damage reported', 'None reported', nan],
      dtype=object)

In [18]:
df.groupby('accident')['accident'].count()

accident
At least 1 accident or damage reported     986
None reported                             2910
Name: accident, dtype: int64

In [23]:
df['accident'] = df['accident'].fillna(df['accident'].mode()[0])

In [38]:
accident_map = {'At least 1 accident or damage reported': 1, 'None reported': 0}
df['accident'] =df['accident'].apply(lambda x : accident_map[x])

In [27]:
df.groupby('clean_title')['clean_title'].count()

clean_title
No      596
Yes    3413
Name: clean_title, dtype: int64

In [26]:
df['clean_title'] = df['clean_title'].fillna('No')

In [12]:
df['brand'].unique()

array(['Ford', 'Hyundai', 'Lexus', 'INFINITI', 'Audi', 'Acura', 'BMW',
       'Tesla', 'Land', 'Aston', 'Toyota', 'Lincoln', 'Jaguar',
       'Mercedes-Benz', 'Dodge', 'Nissan', 'Genesis', 'Chevrolet', 'Kia',
       'Jeep', 'Bentley', 'Honda', 'Lucid', 'MINI', 'Porsche', 'Hummer',
       'Chrysler', 'Volvo', 'Cadillac', 'Lamborghini', 'Maserati',
       'Volkswagen', 'Subaru', 'Rivian', 'GMC', 'RAM', 'Alfa', 'Ferrari',
       'Scion', 'Mitsubishi', 'Mazda', 'Saturn', 'Bugatti', 'Polestar',
       'Rolls-Royce', 'McLaren', 'Buick', 'Lotus', 'Pontiac', 'FIAT',
       'Karma', 'Saab', 'Mercury', 'Plymouth', 'smart', 'Maybach',
       'Suzuki'], dtype=object)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              4009 non-null   object 
 1   model              4009 non-null   object 
 2   model_year         4009 non-null   int64  
 3   milage             4009 non-null   float64
 4   fuel_type          4009 non-null   object 
 5   engine             4009 non-null   object 
 6   transmission       4009 non-null   object 
 7   ext_col            4009 non-null   object 
 8   int_col            4009 non-null   object 
 9   accident           4009 non-null   int64  
 10  clean_title        4009 non-null   object 
 11  price              4009 non-null   float64
 12  transmission_type  4009 non-null   object 
dtypes: float64(2), int64(2), object(9)
memory usage: 407.3+ KB


# Feature Engineering

In [163]:
df.sample(5)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,transmission_type,engine_litre,hp,cylinders
374,Subaru,Outback 3.6R Touring,2017,100680.0,Gasoline,256.0HP 3.6L Flat 6 Cylinder Engine Gasoline Fuel,A/T,Brown,–,0,Yes,20500.0,Automatic,3.6,256.0,6.0
1527,Hyundai,Genesis 3.8,2015,61300.0,Gasoline,311.0HP 3.8L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Silver,Gray,0,No,20150.0,Automatic,3.8,311.0,6.0
1429,BMW,X5 xDrive35i,2016,89000.0,Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Black,Black,0,Yes,24000.0,Automatic,3.0,300.0,6.0
421,Maserati,Ghibli S Q4,2016,1000.0,Gasoline,404.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Gray,Red,0,Yes,36000.0,Manual,3.0,404.0,6.0
2931,BMW,840 Gran Coupe i xDrive,2020,25793.0,Gasoline,3.0L I6 24V GDI DOHC Turbo,8-Speed Automatic,Black Sapphire Metallic,Black,0,No,52999.0,Automatic,3.0,300.0,6.0


In [33]:
df['transmission_type'] = df['transmission'].apply(lambda x: 'Automatic' if 'a/t' in x.lower() or 'automatic' in x.lower() else 'Manual')

In [35]:
df['milage'] = df['milage'].str.replace(r'[^\d.]', '', regex=True).astype(float)
df['price'] = df['price'].str.replace(r'[^\d.]', '', regex=True).astype(float)

In [95]:
def extract_litre_eng(engine_desc):
    match = re.search(r'(\d+(\.\d+)?)\s*[Ll](?:iter)?', engine_desc)
    return float(match.group(1)) if match else None

df['engine_litre'] = df['engine'].apply(extract_litre_eng)
df['engine_litre'].fillna(df['engine_litre'].mode()[0], inplace=True)

In [127]:
def extract_hp(engine_desc):
    match  = re.search(r'(\d+(\.\d+)?)HP', engine_desc)
    return float(match.group(1)) if match else None

df_hp = pd.DataFrame()
df_hp['engine'] = df['engine']
df_hp['hp'] = df_hp['engine'].apply(extract_hp)
print('Hp Mean : ',df_hp['hp'].mean())
print('Hp Mode : ',df_hp['hp'].mode()[0])

Hp Mean :  332.20649796938454
Hp Mode :  300.0


In [129]:
df['hp'] = df['engine'].apply(extract_hp)
df['hp'].fillna(df['hp'].mode()[0], inplace=True)

In [162]:
def extract_cylinders(engine_desc):
    match = re.search(r'\b([VvIi]\d+)\b', engine_desc)
    if match:
        config = match.group(1)
        return int(config[1:])  

    match = re.search(r'(\d+(\.\d+)?)\s*L', engine_desc)
    if match:
        displacement = float(match.group(1))
        if displacement <= 2.5:
            return 4  
        elif displacement <= 3.8:
            return 6  
        else:
            return 8  
    return None


df['cylinders'] = df['engine'].apply(extract_cylinders)
df['cylinders'].fillna(df['cylinders'].mode()[0], inplace=True)

In [185]:
df['transmission'].unique()

array(['6-Speed A/T', '8-Speed Automatic', 'Automatic', '7-Speed A/T',
       'F', 'A/T', '8-Speed A/T', 'Transmission w/Dual Shift Mode',
       '9-Speed Automatic', '6-Speed M/T', '10-Speed A/T', '9-Speed A/T',
       '5-Speed A/T', '1-Speed A/T', 'Automatic CVT',
       '7-Speed Automatic with Auto-Shift', 'CVT-F', 'M/T',
       '6-Speed Automatic with Auto-Shift', '10-Speed Automatic',
       'CVT Transmission', '4-Speed A/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T',
       '8-Speed Automatic with Auto-Shift', '5-Speed Automatic',
       '8-SPEED AT', '1-Speed Automatic', '5-Speed M/T', 'Manual',
       '6-Speed Manual', 'Variable', '2', '7-Speed Manual',
       'Automatic, 9-Spd 9G-Tronic', 'Automatic, 8-Spd',
       'Automatic, 8-Spd Sport w/Sport & Manual Modes', '–',
       'Auto, 6-Spd w/CmdShft', 'Transmission Overdrive Switch',
       '7-Speed Automatic', 'Automatic, 8-Spd PDK Dual-Clutch',
       'Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Ma

In [184]:
def extract_speed(transmission_desc):
    # Regular expression to capture speed information
    match = re.search(r'Single-Speed|(\d+)\s*(?:-?SPEED|Speed|A/T|M/T|Automatic|Manual|CVT)', transmission_desc)
    if match:
        return float(match.group(1))  # Extract and return the speed as a float
    elif re.search(r'Single-Speed', transmission_desc, re.IGNORECASE):
        return 1  # Return 1 if "Single-Speed" is found
    else:
        return None  # Return None if no speed indication is found

extract_speed('6-Speed A/T')

error: nothing to repeat at position 32

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand              4009 non-null   object 
 1   model              4009 non-null   object 
 2   model_year         4009 non-null   int64  
 3   milage             4009 non-null   float64
 4   fuel_type          4009 non-null   object 
 5   engine             4009 non-null   object 
 6   transmission       4009 non-null   object 
 7   ext_col            4009 non-null   object 
 8   int_col            4009 non-null   object 
 9   accident           4009 non-null   int64  
 10  clean_title        4009 non-null   object 
 11  price              4009 non-null   float64
 12  transmission_type  4009 non-null   object 
 13  engine_litre       4009 non-null   float64
 14  hp                 4009 non-null   float64
dtypes: float64(4), int64(2), object(9)
memory usage: 469.9+ KB


# Model Selection And evaluation

In [106]:
# One-hot encode categorical columns
categorical_cols = ['brand', 'model', 'fuel_type', 'transmission_type', 'ext_col', 'int_col', 'clean_title']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Separate features and target variable
X = df.drop('price', axis=1)
y = df['price']

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose a model and train it
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_absolute_error, mean_squared_error
y_pred = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

# To save the model for later use
import joblib
joblib.dump(model, 'car_price_model.pkl')

MAE: 16674.877967581047
MSE: 18314694548.570057
RMSE: 135331.79430041582


['car_price_model.pkl']